<a href="https://colab.research.google.com/github/Decoding-Data-Science/Omantel/blob/main/5_API_new_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install requests gradio

In [4]:
import os

from google.colab import userdata


groq_api_key = userdata.get("GROQ_API_KEY")

In [5]:
import os
import requests
import gradio as gr



url = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}

LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/Omantel/main/download%20(36).png"

OMANTEL_SYSTEM = """You are Aisha, the Omantel AI assistant for developer enablement.

Style
- Be concise, professional, and action-oriented.
- Structure all answers as: Summary → Steps → Code → Notes.
- Prefer bullet points over long paragraphs.

Domain
- Focus on telecom/customer-care use cases: RAG over KB articles, intent classification, call/chat summarization, ticket triage, PII redaction, and evaluation.
- Show Python examples (requests, FastAPI, Gradio) and Groq (Llama 3.1-8B).
- Use realistic ops/QA metrics when relevant: FRT, AHT, intent accuracy, containment/deflection rate, CSAT, latency (p95), and token usage.

Safety & Privacy
- Do not invent facts; say “I’m not sure” if uncertain.
- Never include real keys, internal URLs, or customer data. Use placeholders.
- Redact PII patterns in examples (MSISDN, IMSI, national IDs, emails).
- Assume all data is confidential; follow least-privilege principles.

Localization
- Default to English. If the user writes in Arabic, reply in Arabic.
- Use Asia/Muscat timezone for dates/times when needed.

Formatting
- Put code in fenced blocks with minimal, runnable examples.
- Include curl when helpful.
- Add clear TODOs for env vars and secrets.
"""

def call_groq(messages, temperature=0.2, max_tokens=600, model="llama-3.1-8b-instant"):
    body = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    resp = requests.post(url, headers=headers, json=body, timeout=60)
    if resp.status_code == 200:
        return resp.json()["choices"][0]["message"]["content"]
    else:
        try:
            return f"Error: {resp.status_code} {resp.json()}"
        except Exception:
            return f"Error: {resp.status_code} {resp.text}"

def groq_chat(message, history):
    messages = [{"role": "system", "content": OMANTEL_SYSTEM}]
    for u, a in history:
        if u:
            messages.append({"role": "user", "content": u})
        if a:
            messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": message})
    return call_groq(messages)

# --- Minimal UI with Submit button ---
with gr.Blocks(title="Omantel Developer Assistant") as app:
    gr.HTML("""
    <style>
      .header-row { display:flex; align-items:center; gap:12px; }
      .header-logo img { max-height:48px; border-radius:8px; }
      .header-title { font-size:20px; font-weight:600; margin:0; }
    </style>
    """)

    with gr.Row(elem_classes=["header-row"]):
        gr.Image(value=LOGO_URL, label=None, show_label=False, interactive=False, height=100, elem_classes=["header-logo"])
        title_box = gr.Textbox(value="Omantel Developer Assistant (Aisha)", label="Title", lines=1)
        header_md = gr.Markdown("### Omantel Developer Assistant (Aisha)", elem_classes=["header-title"])

    def _update_header(t):
        return f"### {t}" if t.strip() else "### Omantel Developer Assistant (Aisha)"
    title_box.change(_update_header, inputs=title_box, outputs=header_md)

    gr.Markdown("Ask about telecom/customer-care AI use cases. Answers follow **Summary → Steps → Code → Notes**.")

    chatbot = gr.Chatbot(height=420)
    user_in = gr.Textbox(placeholder="Type your question…", lines=2, label="Your message")
    submit_btn = gr.Button("Submit", variant="primary")
    history_state = gr.State([])  # list of (user, assistant)

    def on_submit(user_msg, history):
        history = history or []
        if not user_msg or not user_msg.strip():
            return history, gr.update(), ""
        bot_reply = groq_chat(user_msg.strip(), history)
        history.append((user_msg.strip(), bot_reply))
        return history, gr.update(value=history), ""

    submit_btn.click(
        fn=on_submit,
        inputs=[user_in, history_state],
        outputs=[history_state, chatbot, user_in],
        api_name=False
    )

if __name__ == "__main__":
    app.launch()


/tmp/ipython-input-2008478020.py:90: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8893539628513f7127.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
